In [3]:
# Paqueterías necesarias para la solución 
import scipy.io
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.animation as animation
import os
from multiprocessing import Pool # Estudiar esta cosa (Importante)
def difusion_piel(x, y, t, nu, u_init):
    '''
    Resuelve la ecuación de difusión en una malla bidimensional con condiciones de Dirichlet en la entrada
    y Neumann en las fronteras laterales.
    
    Parámetros:
        x (array):      Coordenadas x de la malla.
        y (array):      Coordenadas 
# convert  de la malla.El manejo de la vía aérea representa uno de los aspectos más críticos en la atención de pacientes en situación de emergencia o con alteraciones del estado de conciencia. Garantizar que el oxígeno llegue de forma adecuada a los pulmones y, por ende, al resto del cuerpo, es una prioridad que puede marcar la diferencia entre la vida y la muerte. Para lograrlo, es indispensable asegurar una vía aérea permeable, estableciendo así un punto de partida fundamental para cualquier intervención médica avanzada. Esta necesidad no es reciente; se han encontrado referencias históricas que demuestran cómo desde el antiguo Egipto, hacia el 3500 a.C., ya se realizaban procedimientos rudimentarios como la traqueostomía para tratar obstrucciones de la vía aérea.

En la actualidad, el abordaje del paciente crítico se estructura mediante protocolos como el ABCDE, en el que la "A" (Airway) destaca la evaluación y control inmediato de la vía aérea. Para ello, se consideran tanto maniobras manuales como técnicas asistidas con dispositivos, partiendo de lo más básico hacia lo más invasivo. Además, se realiza una evaluación secundaria que incluye herramientas como el método SAMPLE y, para identificar posibles dificultades, mnemotecnias como MEMOM, que permiten anticipar complicaciones durante el manejo avanzado.

Las técnicas empleadas varían según el estado del paciente y las condiciones clínicas. Desde el correcto posicionamiento, la elevación del mentón o la tracción mandibular, hasta el uso de dispositivos como la cánula orofaríngea, mascarilla laríngea o el tubo endotraqueal, todas forman parte del arsenal terapéutico del personal de salud. En situaciones extremas, técnicas como la cricotirotomía pueden ser la única opción viable. Por ello, es esencial que el profesional esté entrenado tanto en el reconocimiento oportuno de una vía aérea comprometida como en la ejecución eficaz de los procedimientos necesarios para restablecerla.


        t (int):        Número de pasos de tiempo.
        nu (float):     Coeficiente de difusión.
        u_init (float): Valor inicial de la concentración en la entrada.
    Retorna:
        u_ap (array):   Solución de la ecuación de  difusión en la malla.
    '''
    # Inicializar variables
    m, n  = x.shape                                                             # Dimensiones de la malla
    T     = np.linspace(0, 3600*20, t)                                             # Discretización del tiempo
    dt    = T[1] - T[0]                                                         # Paso de tiempo
    u_ap  = np.zeros([m, n, t])                                                 # Matriz para almacenar la solución
    Gamma = np.zeros([m, n, 9])                                                 # Matriz para almacenar Gammas

    # Revisamos algo así como el CFL
    dx_min = np.min(np.sqrt((x[1:, :] - x[:-1, :])**2 + (y[1:, :] - y[:-1, :])**2))
                                                                                # Tamaño mínimo de dx
    alpha = nu*dt/dx_min**2                                                     # Número de Courant
    print(f"Número de Courant aproximado: {alpha:.2e}")
    if alpha > 0.5:                                                             # Si el número de Courant es muy grande
        print("El número de Courant es muy grande. Ajuste el paso de tiempo o la viscosidad.")
        exit()                       
# convert                                            # Salir del programa


    # Condición inicial
    u_ap[:, :, 0] = 0                                                           # Condición Inicial

    # Calcular Gammas
    L = np.vstack([[0], [0], [2*nu*dt], [0], [2*nu*dt]])                         # Operador Diferencial,  va a pilando las listas para que quede más ordenado. 
    for i in range(1, m-1):
        for j in range(1, n-1):
            dx = np.array([
                x[i + 1, j], x[i + 1, j + 1], x[i, j + 1], x[i - 1, j + 1],
                x[i - 1, j], x[i - 1, j - 1], x[i, j - 1], x[i + 1, j - 1]
                ]) - x[i, j]                                                    # Todos los dx para el nodo central
        
            dy = np.array([
                y[i + 1, j], y[i + 1, j + 1], y[i, j + 1], y[i - 1, j + 1],
                y[i - 1, j], y[i - 1, j - 1], y[i, j - 1], y[i + 1, j - 1]
                ]) - y[i, j]                                                    # Todos los dy para el nodo central
        
            M = np.vstack([[dx], [dy], [dx**2], [dx*dy], [dy**2]])              # Se crea la matriz de coeficientes
            M = np.linalg.pinv(M)                                               # Se calcula la pseudoinversa (Morse-Penrose)
            YY = M@L                                                            # Se calculan las primeras Gammas
            Gem = np.concatenate(([-np.sum(YY)], YY.flatten()))                 # Se calculan todas las Gammas
            Gamma[i, j, :] = Gem                                                # Se almacenan las Gammas

    # Resolver la ecuación diferencial
    for k in range(1, t):      
        nucleo  = u_ap[1:-1, 1:-1, k-1]                                         # El nucleo es el nodo central
        pesos   = Gamma[1:-1, 1:-1, :]                                          # Los pesos son las Gammas para el esténcil
        vecinos = np.array([
            pesos[:, :, 0] * nucleo,
            pesos[:, :, 1] * u_ap[2:  , 1:-1, k-1],
            pesos[:, :, 2] * u_ap[2:  , 2:  , k-1],
            pesos[:, :, 3] * u_ap[1:-1, 2:  , k-1],
            pesos[:, :, 4] * u_ap[0:-2, 2:  , k-1],
            pesos[:, :, 5] * u_ap[0:-2, 1:-1, k-1],
            pesos[:, :, 6] * u_ap[0:-2, 0:-2, k-1],
            pesos[:, :, 7] * u_ap[1:-1, 0:-2, k-1],
            pesos[:, :, 8] * u_ap[2:  , 0:-2, k-1],
        ])                                                                     # Se calculan las contribuciones de los vecinos
        u_ap[1:-1, 1:-1, k] = nucleo + np.sum(vecinos, axis = 0)               # Se calcula el nuevo paso de tiempo

        # Condiciones de frontera de Dirichlet en la entrada
        u_ap[:,   0, k] = u_init                                               # Condición de Dirichlet en la entrada

        # Condiciones de Neumann (no
# convert  flujo) en las fronteras laterales
        # Calculadas de forma clásica con en Diferencias Finitas Clásicas
        u_ap[:,   n-1, k] = u_ap[:,   n-2, k]                                  # Condición de Neumann en la pared inferior
        u_ap[m-1, :,   k] = u_ap[m-2, :,   k]                                  # Condición de Neumann en la pared izquierda
        u_ap[0,   :,   k] = u_ap[1,   :,   k]                                  # Condición de Neumann en la pared derecha
    
    return u_ap
# Generación de la base de datos 

# Cargar datos de la malla
datos  = scipy.io.loadmat('piel224.mat')

# Definir los parámetros del problem (esto es lo que va a variar en el problema)
x, y   = datos["x"], datos["y"]
t      = 500
output_dir = 'skin224'
# Generación de la base de datos 
# Resolver la ecuación de difusión y crear el gif
  # === Función del proceso paralelo ===
def run_case(params):
    ss, j = params
    nu = ss * 1e-9
    u_init = j

    print(f"Simulando para Ci={j}, nu={nu}...")
    u_ap = difusion_piel(x, y, t, nu, u_init)

    if u_ap is None:
        return

    # Crear carpeta para el valor actual de nu
    carpeta_nu = os.path.join(output_dir, f"nu_{nu:.9f}")
    os.makedirs(carpeta_nu, exist_ok=True)  # Asegura que la carpeta exista

    # Crear y guardar el gráfico
    plt.figure(figsize=(8, 6))
    plt.pcolormesh(x, y, u_ap[:, :, t - 1], shading='auto', cmap='cool_r',
                   facecolor='lightgray', vmin=0, vmax=900)
    plt.axis('off')  # Oculta todos los ejes y ticks

    filename = os.path.join(carpeta_nu, f"c_{j:03d}.png")  # Formato: c_001.png
    plt.savefig(filename, dpi=75, bbox_inches='tight', pad_inches=0)

    plt.close()
    print(f"Guardado: {filename}")

# === Ejecutar en paralelo ===
if __name__ == "__main__":
    combinations = [(i, j) for i in range(1, 201) for j in range(1, 901)]
    with Pool(processes=1) as pool:
        pool.map(run_case, combinations)



Simulando para Ci=1, nu=1e-09...
Número de Courant aproximado: 5.60e-03


/tmp/ipykernel_40951/3018537923.py:127: UserWarning: The input coordinates to pcolormesh are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolormesh.
  plt.pcolormesh(x, y, u_ap[:, :, t - 1], shading='auto', cmap='cool_r',


Guardado: skin224/nu_0.000000001/c_001.png
Simulando para Ci=2, nu=1e-09...
Número de Courant aproximado: 5.60e-03
Guardado: skin224/nu_0.000000001/c_002.png
Simulando para Ci=3, nu=1e-09...
Número de Courant aproximado: 5.60e-03
Guardado: skin224/nu_0.000000001/c_003.png
Simulando para Ci=4, nu=1e-09...
Número de Courant aproximado: 5.60e-03
Guardado: skin224/nu_0.000000001/c_004.png
Simulando para Ci=5, nu=1e-09...
Número de Courant aproximado: 5.60e-03
Guardado: skin224/nu_0.000000001/c_005.png
Simulando para Ci=6, nu=1e-09...
Número de Courant aproximado: 5.60e-03
Guardado: skin224/nu_0.000000001/c_006.png
Simulando para Ci=7, nu=1e-09...
Número de Courant aproximado: 5.60e-03
Guardado: skin224/nu_0.000000001/c_007.png
Simulando para Ci=8, nu=1e-09...
Número de Courant aproximado: 5.60e-03
Guardado: skin224/nu_0.000000001/c_008.png
Simulando para Ci=9, nu=1e-09...
Número de Courant aproximado: 5.60e-03
Guardado: skin224/nu_0.000000001/c_009.png
Simulando para Ci=10, nu=1e-09...
Núm

Process ForkPoolWorker-2:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.10/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/usr/lib/python3.10/multiprocessing/pool.py", line 48, in mapstar
    return list(map(*args))
  File "/tmp/ipykernel_40951/3018537923.py", line 116, in run_case
    u_ap = difusion_piel(x, y, t, nu, u_init)
  File "/tmp/ipykernel_40951/3018537923.py", line 64, in difusion_piel
    M = np.vstack([[dx], [dy], [dx**2], [dx*dy], [dy**2]])              # Se crea la matriz de coeficientes
  File "/home/letmitaf/.local/lib/python3.10/site-packages/numpy/core/shape_base.py", line 286, in vstack
    arrs = atleast_2d(*tup)
  File "/home/letmitaf/.local/lib/python3.10/site-packages/numpy/core/shape_base

KeyboardInterrupt: 